# ConvLSTM

## Requirements
- python 3.7
- keras
- tensorflow
- scikit-learn
- matplotlib
- seaborn
- pandas
- numpy
- wandb

## Imports

In [1]:
import pandas as pd
import numpy as np
import os


# PlaidML Imports - disable if you make use of cudnn
amd = True

if amd:
    import plaidml.keras
    plaidml.keras.install_backend()
    os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
#     os.environ['PLAIDML_USE_STRIPE']='1'
#     os.environ['PLAIDML_VERBOSE']='1'

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import wandb
from wandb.keras import WandbCallback

from keras.models import Sequential, load_model
from keras.layers import Dense, ConvLSTM2D, LSTM, Dropout, Flatten, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1, l2

wandb.init(project="convlstm")

wandb: WARNING Keras version 2.2.4 is not fully supported. Required keras >= 2.4.0
wandb: Currently logged in as: tyromijn (use `wandb login --relogin` to force relogin)


## Methods

In [2]:
def to_supervised(train, n_input, n_output=8):
    X, y = list(),list()
    
    X_start = 0
    
    # iterate over train dataset
    for _ in range(len(train)):
        
        # set the ranges for input + output
        X_end = X_start + n_input
        y_end = X_end + n_output
        
        # check if data contains enough samples for sequence
        if y_end <= len(train):
            X.append(train[X_start:X_end, :])
            y.append(train[X_end:y_end, 0])    
        X_start += 1
    return np.array(X), np.array(y)

def build_model(data, n_steps, n_length, n_input, params):
    
    # ConvLSTM2D expected input: [samples, timesteps, rows, cols, channels]
    # In our case, this will be [n_input, 7, 1, 24, 1]
    
    # data preperation
    train, val = data
    X_train, y_train = to_supervised(train, n_input)
    X_val, y_val = to_supervised(val, n_input)
    
    # meta / parameters
    epochs, batch_size, verbose, learning_rate, n_checkpoint, activation_function = params
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    
    X_train = X_train.reshape((X_train.shape[0], n_steps, 1, n_length, n_features))
    X_val = X_val.reshape((X_val.shape[0], n_steps, 1, n_length, n_features))
    
    # reshape output
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], 1))
    
    model = Sequential()
    model.add(ConvLSTM2D(filters=8, kernel_size=(5,5), activation=activation_function, input_shape=(n_steps, 1, n_length, n_features), return_sequences=True))
    model.add(ConvLSTM2D(filters=8, kernel_size=(5,5), activation=activation_function, return_sequences=True))
    model.add(Flatten())
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(128, activation=activation_function, return_sequences=True))
    model.add(TimeDistributed(Dense(128, activation=activation_function)))
    model.add(TimeDistributed(Dense(1)))
    opt = Adam(lr=learning_rate)
    model.compile(loss='mse', optimizer=opt)
    checkpoint = ModelCheckpoint("convmodelcheckpoint.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=n_checkpoint)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=(X_val, y_val), callbacks=[WandbCallback(), checkpoint], shuffle=False)
    
    model.save(os.path.join(wandb.run.dir, "model.h5"))
    
    # plot the model
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    
    return model

def forecast(model, history, n_steps, n_length, n_input, n_features):
    data = np.array(history)
    X = data[-n_input:, 0]
    X = X.reshape((1, n_steps, 1, n_length, n_features))
    yhat = model.predict(X, verbose=0)
    yhat = yhat[0]
    return yhat


def visualize_forecast(history, validation, future, pred, future_pred, n_input, n_forecasts):
    if n_input > 48:
        past = history[-48:-n_forecasts+1]
    else:   
        past = history[-n_input:-n_forecasts+1]
    
    future = future[-n_forecasts:]
    validation = validation[-n_forecasts:]

#     validation.index = validation.index + pd.Timedelta(n_forecasts, unit='h')
    future.index = future.index + pd.Timedelta(n_forecasts, unit='h')

    future['Prediction'] = future_pred
    validation['Prediction'] = pred
    validation['Actual'] = history[-8:]['Global_active_power']

    
    sns.set(rc={'figure.figsize':(14,5)})
    sns.lineplot(data=past, x=past.index, y="Global_active_power")
    sns.lineplot(data=validation, x=validation.index, y="Actual", color="lime")
    sns.lineplot(data=validation, x=validation.index, y="Prediction", color="red")
    sns.lineplot(data=future, x=future.index, y="Prediction", color="darkred")
    plt.legend(['Active Power (Past)','Active Power (Actual)','Active Power (Prediction)', 'Active Power (Future Prediction)'])

    plt.title('Predictions of Global Active Power')
    plt.xlabel('Timeline')
    plt.ylabel('Global Active Power')
    plt.grid(which='major', color="#ffffff", alpha=.5)
    plt.axvline(x=past.index[-1], color="green", linestyle="--")
    plt.axvline(x=validation.index[-1], color="green", linestyle="--")

## Data Analysis / Preperation

In [3]:
df = pd.read_csv('data/household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

df_resample = pd.DataFrame(columns=df.columns)

# global_active_power: household global minute-averaged active power (in kilowatt)
# global_reactive_power: household global minute-averaged reactive power (in kilowatt)
# voltage: minute-averaged voltage (in volt)
# global_intensity: household global minute-averaged current intensity (in ampere)

# df_resample['Global_active_power'] = np.cumsum(df['Global_active_power'].resample('h').mean())
# df_resample['Global_reactive_power'] = np.cumsum(df['Global_reactive_power'].resample('h').mean())
df_resample['Global_active_power'] = df['Global_active_power'].resample('h').mean()
df_resample['Global_reactive_power'] = df['Global_reactive_power'].resample('h').mean()
df_resample['Voltage'] = df['Voltage'].resample('h').mean()
df_resample['Global_intensity'] = df['Global_intensity'].resample('h').mean()

# sub_metering_#: energy sub-metering No. # (in watt-hour of active energy).
df_resample['Sub_metering_1'] = df['Sub_metering_1'].resample('h').sum()
df_resample['Sub_metering_2'] = df['Sub_metering_2'].resample('h').sum()
df_resample['Sub_metering_3'] = df['Sub_metering_3'].resample('h').sum()

# columns = [x for x in df_resample.columns if x not in ['Voltage', 'Global_intensity', 'Global_reactive_power']] 
columns = [x for x in df_resample.columns if x not in ['Voltage', 'Global_intensity']] 

df_resample = df_resample[columns]

df = df_resample

df_resample.head()

,Global_active_power,Global_reactive_power,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,
2006-12-16 17:00:00,4.222889,0.229000,0.0,19.0,607.0
2006-12-16 18:00:00,3.632200,0.080033,0.0,403.0,1012.0
2006-12-16 19:00:00,3.400233,0.085233,0.0,86.0,1001.0
2006-12-16 20:00:00,3.268567,0.075100,0.0,0.0,1007.0
2006-12-16 21:00:00,3.056467,0.076667,0.0,25.0,1033.0


In [4]:
# Fill NaN's with column's mean value
for j in range(0,len(df_resample.columns)):        
        df_resample.iloc[:,j]=df_resample.iloc[:,j].fillna(df_resample.iloc[:,j].mean())

In [5]:
df_resample

,Global_active_power,Global_reactive_power,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,
2006-12-16 17:00:00,4.222889,0.229000,0.0,19.0,607.0
2006-12-16 18:00:00,3.632200,0.080033,0.0,403.0,1012.0
2006-12-16 19:00:00,3.400233,0.085233,0.0,86.0,1001.0
2006-12-16 20:00:00,3.268567,0.075100,0.0,0.0,1007.0
2006-12-16 21:00:00,3.056467,0.076667,0.0,25.0,1033.0
...,...,...,...,...,...
2010-11-26 17:00:00,1.725900,0.061400,0.0,0.0,772.0
2010-11-26 18:00:00,1.573467,0.053700,0.0,0.0,0.0
2010-11-26 19:00:00,1.659333,0.060033,0.0,4.0,0.0


## Model building

In [6]:
# no. hours to forecast
n_forecasts = 8

# Amount of sequences that the data will be split into
n_steps = 21

# Amount of hours per sequence
n_length = 24

# Amount of features in dataset
n_features = len(df_resample.columns)

# determines how much data is used to forecast (currently using a week of data per row)
n_input = n_steps * n_length

# determines whether we're going to train or evaluate an existing model
training = True

In [7]:
# remove a few hours of time, validate if model isn't inaccurate due to an anomaly in the data
# df_resample = df_resample[:-8]

df_future = df_resample[-n_forecasts:]
df_pred = df_resample[-n_input*n_features:]
df_resample = df_resample[:-n_forecasts]

train, test = train_test_split(df_resample.values, test_size=.2, shuffle=False, stratify=None)

In [8]:
if training:
    data = [train, test]

    # params = [epochs, batch_size, verbose, learning_rate, n_checkpoint, activation_function]
    params = [750, 512, 1, 0.001, 25, 'tanh']

    model = build_model(data, n_steps, n_length, n_input, params)

else:
    model = load_model('convmodelcheckpoint.hdf5')

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Train on 27153 samples, validate on 6406 samples
Epoch 1/750


INFO:plaidml:Analyzing Ops: 729 of 3024 operations complete
INFO:plaidml:Analyzing Ops: 2314 of 3024 operations complete


27136/27153 [============================>.] - ETA: 0s - loss: 1.2805

INFO:plaidml:Analyzing Ops: 615 of 3024 operations complete
INFO:plaidml:Analyzing Ops: 2245 of 3024 operations complete
INFO:plaidml:Analyzing Ops: 684 of 1285 operations complete
INFO:plaidml:Analyzing Ops: 768 of 1285 operations complete


27153/27153 [==============================] - 80s 3ms/step - loss: 1.2800 - val_loss: 0.5787
Epoch 2/750
27153/27153 [==============================] - 16s 597us/step - loss: 0.9423 - val_loss: 0.6311
Epoch 3/750
27153/27153 [==============================] - 17s 610us/step - loss: 0.9484 - val_loss: 0.6493
Epoch 4/750
27153/27153 [==============================] - 17s 634us/step - loss: 0.9360 - val_loss: 0.5063
Epoch 5/750
27153/27153 [==============================] - 17s 638us/step - loss: 0.8648 - val_loss: 0.5356
Epoch 6/750
27153/27153 [==============================] - 17s 640us/step - loss: 0.8375 - val_loss: 0.5185
Epoch 7/750
27153/27153 [==============================] - 16s 596us/step - loss: 0.8044 - val_loss: 0.5018
Epoch 8/750
27153/27153 [==============================] - 16s 593us/step - loss: 0.7721 - val_loss: 0.5048
Epoch 9/750
27153/27153 [==============================] - 16s 592us/step - loss: 0.7359 - val_loss: 0.5122
Epoch 10/750
27153/27153 [================

KeyboardInterrupt: 

## Forecasting & Visualizing

In [9]:
history = np.array([x for x in train])

# Forecast
pred = forecast(model, history, n_steps, n_length, n_input*n_features, n_features)
future_pred = forecast(model, df_pred, n_steps, n_length, n_input*n_features, n_features)

# Visualize
visualize_forecast(df, df_pred, df_future, pred, future_pred, n_input, n_forecasts)

NameError: name 'model' is not defined